In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, Conv2D, LeakyReLU, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Ruta al directorio de imágenes descargadas
directorio_imagenes = "/content/drive/MyDrive/dataset" # Reemplaza con el directorio correcto

# Lista para almacenar las imágenes y las etiquetas
imagenes = []
etiquetas = []

# Recorre el directorio de imágenes y carga las imágenes en la lista
for nombre_archivo in os.listdir(directorio_imagenes):
    if nombre_archivo.endswith(".jpg"):
        ruta_imagen = os.path.join(directorio_imagenes, nombre_archivo)

        # Verifica si la imagen se puede leer correctamente
        imagen = cv2.imread(ruta_imagen)
        if imagen is None:
            print(f"Error al leer la imagen: {ruta_imagen}")
            continue

        # Verifica si la imagen tiene dimensiones válidas antes de redimensionarla
        if imagen.shape[0] < 32 or imagen.shape[1] < 32:
            print(f"Dimensiones inválidas para la imagen: {ruta_imagen}")
            continue

        imagen = cv2.resize(imagen, (32, 32)) # Redimensiona la imagen a 32x32 píxeles
        imagenes.append(imagen)
        etiquetas.append(1)  # Reemplaza esto con la etiqueta adecuada para cada imagen

# Convierte las listas en matrices numpy
X = np.array(imagenes)
y = np.array(etiquetas)

# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normaliza los valores de píxeles
trainX = X_train / 255.0
testX = X_test / 255.0

# Ajusta las dimensiones de las matrices de etiquetas
trainy = np.reshape(y_train, (-1, 1))
testy = np.reshape(y_test, (-1, 1))

# Definir el discriminador
def define_discriminator(in_shape=(32, 32, 3)):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # Compilar el modelo
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Definir el generador
def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 256 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 256)))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))
    return model

# Definir el modelo GAN
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# Tamaño del espacio latente
latent_dim = 100

# Crear el discriminador
discriminator = define_discriminator()
discriminator.summary()

# Crear el generador
generator = define_generator(latent_dim)
generator.summary()

# Crear el modelo GAN
gan_model = define_gan(generator, discriminator)
gan_model.summary()

# Entrenamiento

# Configuración de hiperparámetros
n_epochs = 100
n_batch = 128
bat_per_epo = int(trainX.shape[0] / n_batch)

# Carga y preprocesa los datos de entrenamiento utilizando tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((trainX, trainy))
train_dataset = train_dataset.shuffle(buffer_size=trainX.shape[0]).batch(n_batch)

# Bucle de entrenamiento
for epoch in range(n_epochs):
    for batch, (X_real, y_real) in enumerate(train_dataset):
        # Entrenar el discriminador en datos reales
        d_loss1, _ = discriminator.train_on_batch(X_real, y_real)

        # Generar datos falsos
        X_gan = tf.random.normal(shape=(n_batch, latent_dim))
        X_fake = generator.predict(X_gan)
        y_fake = np.zeros((n_batch, 1))

        # Entrenar el discriminador en datos falsos
        d_loss2, _ = discriminator.train_on_batch(X_fake, y_fake)

        # Entrenar el generador
        X_gan = tf.random.normal(shape=(n_batch, latent_dim))
        y_gan = np.ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)

        # Imprimir progreso
        print(f"Epoch: {epoch+1}/{n_epochs}, Batch: {batch}/{bat_per_epo}, D_loss_real: {d_loss1}, D_loss_fake: {d_loss2}, G_loss: {g_loss}")

# Guardar el generador
generator.save('/content/drive/MyDrive/generator_model.h5')


Epoch: 29/100, Batch: 17/23, D_loss_real: 0.4669876992702484, D_loss_fake: 0.5600870847702026, G_loss: 1.5834075212478638
4/4 [==============================] - 1s 240ms/step
Epoch: 29/100, Batch: 18/23, D_loss_real: 0.6027494668960571, D_loss_fake: 0.5805165767669678, G_loss: 1.4997116327285767
4/4 [==============================] - 1s 241ms/step
Epoch: 29/100, Batch: 19/23, D_loss_real: 0.5684380531311035, D_loss_fake: 0.6257758140563965, G_loss: 1.5934300422668457
4/4 [==============================] - 1s 236ms/step
Epoch: 29/100, Batch: 20/23, D_loss_real: 0.5282111763954163, D_loss_fake: 0.6098830103874207, G_loss: 1.9433903694152832
4/4 [==============================] - 1s 239ms/step
Epoch: 29/100, Batch: 21/23, D_loss_real: 0.5676002502441406, D_loss_fake: 0.5091761350631714, G_loss: 2.064661741256714
4/4 [==============================] - 1s 235ms/step
Epoch: 29/100, Batch: 22/23, D_loss_real: 0.6345123052597046, D_loss_fake: 0.585532546043396, G_loss: 2.1063284873962402
4/4 [

In [ ]:
# Guardar el generador
generator.save('/content/drive/MyDrive/generator_model.h5')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Cargar el modelo del generador
generator = load_model('/content/drive/MyDrive/generator_model.h5')

# Tamaño del espacio latente
latent_dim = 100

# Generar muestras latentes aleatorias
latent_points = np.random.normal(size=(5, latent_dim))

# Generar imágenes falsas
fake_images = generator.predict(latent_points)
fake_images = (fake_images + 1) / 2.0

# Mostrar las imágenes generadas
fig, axes = plt.subplots(1, 5, figsize=(10, 2))
axes = axes.flatten()

for i, ax in enumerate(axes):
    ax.axis('off')
    ax.imshow(fake_images[i])

plt.tight_layout()
plt.show()
